In [1]:
!pip install funcy

In [1]:
import numpy as np

from funcy  import *
from neal   import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder

In [13]:
BETA_RANGE         = (5, 100)  # 焼きなましの温度の逆数。大きい方が解が安定しますが、局所解に陥る可能性が高くなってしまいます
NUM_READS          = 1000        # 焼きなましする回数。NUM_READS個の解が生成されます。多いほうが良い解がでる可能性が高くなります
NUM_SWEEPS         = 100000    # 焼きなましのステップを実施する回数。1つの解を生成するために繰り返し処理をする回数です。大きい方が良い解がでる可能性が高くなります
BETA_SCHEDULE_TYPE = 'linear'  # 焼きなましの温度をどのように変化させるか。linearだと線形に変化させます

def calc_pyqubo(bqm):
    import neal
    sa = neal.SimulatedAnnealingSampler()
    sampleset = sa.sample(bqm, beta_range=BETA_RANGE, 
                          num_reads=NUM_READS, num_sweeps=NUM_SWEEPS, 
                          beta_schedule_type=BETA_SCHEDULE_TYPE, seed=1)
    # Decode solution
    decoded_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
    best_sample = min(decoded_samples, key=lambda x: x.energy)
    num_broken = len(best_sample.constraints(only_broken=True))
    print("number of broken constarint = {}".format(num_broken))
    return best_sample.sample

In [14]:
M = 5   # 社員の数
D = 5  # 日数

# QUBOを構成する変数を定義します
xs = Array.create('x', shape=(M, D), vartype='BINARY')

# チューニングのための変数を定義します
a = Placeholder('A')
b = Placeholder('B')

# QUBOを定義します。ここから……
h = 0

# 1日に2名以上、かつ、できるだけ少なくという制約を追加します。2名より多くても少なくてもペナルティが発生するようになっています
for d in range(D):
    # 2を引くと、少なければ負、多ければ正の数になるわけですが、それを2乗して正の値にします
    h += a * Constraint((sum(xs[m][d] for m in range(M)) - 2) ** 2, f'day-{d}') 
# 同じ人と別の日に出社しないという制約を追加します
for m1 in range(M):
    for m2 in range(m1 + 1, M):
        for d1 in range(D):
            for d2 in range(d1 + 1, D):
                # xsは1か0なので、掛け算をする場合は、全部1の場合にだけ1になりますのでペナルティとなります。
                # 要は2つの日で同じひとと出勤したときのみ以下の式は1となります
                h += b * xs[m1][d1] * xs[m2][d1] * xs[m1][d2] * xs[m2][d2] 

In [15]:
# コンパイルしてモデルを作ります
model = h.compile()
# ……ここまで。QUBOを定義します

# チューニングのための変数の値
feed_dict = {'A': 2.0, 'B': 1.0}

# イジング模型を生成して、nealで解きます
bqm = model.to_bqm(feed_dict=feed_dict)
qubo = model.to_qubo(feed_dict=feed_dict)

In [16]:
def get_energy(Q,sol):
    return sum (sol[q[0]] * sol[q[1]] * Q[q] for q in Q)

sol = calc_pyqubo(bqm)
Q = qubo[0]
print('エネルギー値=',get_energy(Q,sol))

number of broken constarint = 0
エネルギー値= -40.0


In [20]:
def gen_str_arr(s,i,j):
    return s+'['+str(i)+']['+str(j)+']'


if sol:
    mat = np.full((M, D), -1)
    city_order = []
    for i in range(M):
        for j in range(D):
            mat[i,j] =  sol[gen_str_arr('x', i, j)]
            if sol[gen_str_arr('x', i, j)] == 1:
                city_order.append(j)
    print(mat)
    print(city_order)

[[0 0 0 0 1]
 [1 0 0 1 0]
 [1 0 1 0 1]
 [0 1 1 0 0]
 [0 1 0 1 0]]
[4, 0, 3, 0, 2, 4, 1, 2, 1, 3]


In [21]:
# 日単位で、出社する社員を出力します
for d in range(D):
    print(tuple(keep(lambda m: 'ABCDE'[mat] if sol[gen_str_arr('x', m, d)] == 1 else False, range(M))))

TypeError: only integer scalar arrays can be converted to a scalar index

In [19]:
sol

{'x[4][4] * x[2][4]': 0,
 'x[4][3] * x[2][3]': 0,
 'x[4][3]': 1,
 'x[4][2] * x[2][2]': 0,
 'x[4][2] * x[1][2]': 0,
 'x[4][1] * x[1][1]': 0,
 'x[4][4] * x[1][4]': 0,
 'x[4][1]': 1,
 'x[3][4] * x[2][4]': 0,
 'x[3][4] * x[1][4]': 0,
 'x[3][4] * x[4][4]': 0,
 'x[3][4] * x[0][4]': 0,
 'x[4][1] * x[0][1]': 0,
 'x[3][4]': 0,
 'x[4][3] * x[0][3]': 0,
 'x[4][0] * x[2][0]': 0,
 'x[3][3] * x[4][3]': 0,
 'x[3][3] * x[2][3]': 0,
 'x[3][3]': 0,
 'x[3][2] * x[2][2]': 1,
 'x[3][2] * x[4][2]': 0,
 'x[4][0] * x[1][0]': 0,
 'x[3][2] * x[0][2]': 0,
 'x[3][2]': 1,
 'x[3][1] * x[2][1]': 0,
 'x[3][0]': 0,
 'x[3][1] * x[1][1]': 0,
 'x[3][0] * x[2][0]': 0,
 'x[2][3] * x[0][3]': 0,
 'x[2][2] * x[1][2]': 0,
 'x[3][3] * x[0][3]': 0,
 'x[2][2] * x[0][2]': 0,
 'x[2][2]': 1,
 'x[4][0]': 0,
 'x[3][0] * x[0][0]': 0,
 'x[4][0] * x[0][0]': 0,
 'x[2][1] * x[1][1]': 0,
 'x[4][2]': 0,
 'x[3][3] * x[1][3]': 0,
 'x[2][1] * x[0][1]': 0,
 'x[4][4]': 0,
 'x[2][0] * x[0][0]': 0,
 'x[3][2] * x[1][2]': 0,
 'x[2][3] * x[1][3]': 0,
